In [ ]:
from playwright.async_api import async_playwright
from playwright.async_api import TimeoutError as PlaywrightTimeoutError
import re
import pandas as pd
import jquantsapi
from dotenv import load_dotenv
import os
from tqdm import tqdm

In [ ]:
load_dotenv()
my_mail_address:str = os.environ['JQUANTS_EMAIL']
my_password: str = os.environ['JQUANTS_PASSWORD']
cli = jquantsapi.Client(mail_address=my_mail_address, password=my_password)

In [ ]:
jq_info = cli.get_listed_info()

In [ ]:
# Playwrightインスタンスを作成
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
search_page = await browser.new_page()

# EDINETにアクセス
await search_page.goto('https://disclosure2.edinet-fsa.go.jp/WEEK0010.aspx')

In [ ]:
def parse_financial_value(text):
    """
    財務諸表の値をパースして数値に変換
    
    Args:
        text: "※3 △1,234\n" や "※１,※２ 5,288,322" のような形式の文字列
    
    Returns:
        int または None: パースした数値、ハイフンの場合はNone
    """
    if not text:
        return None
    
    # 改行を削除してトリム
    text = text.strip()
    
    # ハイフンのみの場合はNoneを返す（複数の注釈記号にも対応）
    if re.match(r'^(?:※[０-９\d]+(?:[,、]\s*※[０-９\d]+)*\s*)?[‐－−-]$', text):
        return None
    
    # パターン: [複数の※記号] [△][カンマ付き数値]
    # 複数の注釈記号（※１,※２など）、△（任意）、数値部分を抽出
    # 全角数字（０-９）と半角数字（\d）の両方に対応
    match = re.search(r'(?:※[０-９\d]+(?:[,、]\s*※[０-９\d]+)*\s*)?(△)?([\d,]+)', text)
    
    if not match:
        return None
    
    # 三角形の有無と数値部分を取得
    is_negative = match.group(1) is not None
    number_str = match.group(2)
    
    # カンマを削除して数値に変換
    number = int(number_str.replace(',', ''))
    
    # 三角形がある場合は負の値にする
    if is_negative:
        number = -number
    
    return number

In [ ]:
data = []

i = 0
for security_code in jq_info['Code']:

    # デバッグ用：
    # security_code = '1301'
    
    # デバッグ用：50銘柄で終了
    if i >= 100:
        break
    else:
        i += 1
    
    # 証券コード
    security_code = security_code[:4]

    # 証券コードを入力してEnterキーを押す
    await search_page.get_by_role("textbox", name="D_Keyword").fill(security_code)
    await search_page.get_by_role("textbox", name="D_Keyword").press("Enter")
    
    await search_page.wait_for_timeout(1000)
    try:
        await search_page.wait_for_load_state('networkidle', timeout=5000)
    except PlaywrightTimeoutError:
        pass
        
    
    # 「有価証券報告書」のリンクが存在するか確認
    report_link = search_page.get_by_role("link", name=re.compile("^有価証券報告書")).first

    # リンクが存在しない場合はスキップ
    if await report_link.count() == 0:
        print(f"証券コード {security_code}: 有価証券報告書が見つかりません。スキップします。")
        continue

    # 「有価証券報告書」で始まる最初のリンクをクリック
    async with search_page.expect_popup() as popup_info:
        await report_link.click()
    
    report_page = await popup_info.value
    
    await report_page.wait_for_timeout(1000)
    try:
        await report_page.wait_for_load_state('networkidle', timeout=5000)
    except PlaywrightTimeoutError:
        pass

    # 連結賃借対照表があるか確認
    mokuji_frame = report_page.locator("iframe[name=\"frame_mokuji\"]").content_frame
    consolidated_bs_link = mokuji_frame.get_by_role("link", name="連結貸借対照表", exact=True)
    standalone_bs_link = mokuji_frame.get_by_role("link", name="貸借対照表", exact=True)
    if await consolidated_bs_link.count() > 0:
        # 存在する場合はクリック
        await consolidated_bs_link.click()
    else:
        # 賃借対照表をクリック
        await standalone_bs_link.click()

    await report_page.wait_for_timeout(1000)
    try:
        await report_page.wait_for_load_state('networkidle', timeout=5000)
    except PlaywrightTimeoutError:
        pass

    # 項目を取得
    item_keys = ['資産合計', '純資産合計', '現金及び預金', '預金', '土地', '投資有価証券', '短期借入金', '１年内返済予定の長期借入金', '長期借入金']
    items = {'証券コード': security_code}
    frame = report_page.locator("iframe[name=\"frame_honbun\"]").content_frame
    for item_key in item_keys:
        locator = frame.locator(f'//tr[.//text()="{item_key}"]/td[last()]')
        if await locator.count() > 0:
            value = ''.join(await locator.all_text_contents())
            items[item_key] = parse_financial_value(value)
        else:
            items[item_key] = None  # または適切なデフォルト値
    data.append(items)
    
    await report_page.close()

    # デバッグ用
    # break

In [ ]:
parse_financial_value(value)

In [ ]:
# 使い終わったらブラウザを閉じる
await browser.close()
await playwright.stop()

In [ ]:
df

In [ ]:
df = pd.DataFrame(data)
df['自己資本比率'] = df['純資産合計'] / df['資産合計']
df['ネットキャッシュ'] = (df['現金及び預金'] + df['預金'] + df['投資有価証券']) - (df['短期借入金'] + df['長期借入金'] + df['１年内返済予定の長期借入金'])
df